# Exploratory data analysis

In [291]:
from typing import Dict

from scipy.stats import ks_2samp
import pandas as pd
import numpy as np
from plotly import express as px
import os
import re
from IPython.display import display_markdown

In [3]:
for d in os.listdir('../data/raw'):
    print(d)

.gitkeep
data_description.txt
sample_submission.csv
test.csv
train.csv


In [24]:
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')

train['_t'] = 'train'
test['_t'] = 'test'

In [25]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,_t
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,train
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,train
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,train
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,train
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,train


In [232]:
def get_descriptions() -> Dict[str, str]:
    descriptions: Dict[str, str] = {}
    feature_name_pattern = re.compile('^(\w+):.*')

    with open('../data/raw/data_description.txt', 'r') as f:        
        lines: list[str] = f.readlines()
        current_feature_name: str = ''
        current_feature_summary: str = ''
        current_feature_desc: str = ''

        for line in lines:
            match = feature_name_pattern.match(line)
            if match:
                tmp: str = match.group(1)
                if current_feature_name != tmp and current_feature_name != '':
                    descriptions[current_feature_name] = { 'summary': current_feature_summary, 'description': current_feature_desc }
                current_feature_name = tmp
                current_feature_desc = line
                current_feature_summary = line
            else:
                current_feature_desc += line
        
        descriptions[current_feature_name] = { 'summary': current_feature_summary, 'description': current_feature_desc }
    return descriptions

descriptions = get_descriptions()

In [75]:
print(descriptions['Alley']['summary'])

Alley: Type of alley access to property



## Features

### Sale price

Рассмотрим распределение цен на недивжимость в обучающей выборке.

In [35]:
px.histogram(train['SalePrice'], nbins=100, title='Sale Price')

In [23]:
px.histogram(train['SalePrice'].apply(lambda x: np.log(x)), nbins=100, title='Sale Price (log)')

Возможно, лучше будет работать predict для `log(target)` и на постпроцессинге делать обратное преобразование.

### Остальные признаки

In [263]:
cat_features = train.select_dtypes([np.object]).columns
md: str = f'{len(cat_features)} категориальных признаков (в том числе технический признак `_t`):\n\n'
md += '| № | признак | описание | nunique | # n/a | # not n/a |\n'
md += '|--:|:-------:|:---------|--------:|------:|----------:|\n'
for i, cat_feature in enumerate(cat_features):
    if cat_feature == '_t':
        continue

    md += f'| {i + 1} | **{cat_feature}** | {descriptions[cat_feature]["summary"].split(":")[1].strip()} | {train[cat_feature].nunique()} | {train[train[cat_feature].isna()].shape[0]} | {train[~train[cat_feature].isna()].shape[0]} |\n'

display_markdown(md, raw=True)

44 категориальных признаков (в том числе технический признак `_t`):

| № | признак | описание | nunique | # n/a | # not n/a |
|--:|:-------:|:---------|--------:|------:|----------:|
| 1 | **MSZoning** | Identifies the general zoning classification of the sale. | 5 | 0 | 1460 |
| 2 | **Street** | Type of road access to property | 2 | 0 | 1460 |
| 3 | **Alley** | Type of alley access to property | 2 | 1369 | 91 |
| 4 | **LotShape** | General shape of property | 4 | 0 | 1460 |
| 5 | **LandContour** | Flatness of the property | 4 | 0 | 1460 |
| 6 | **Utilities** | Type of utilities available | 2 | 0 | 1460 |
| 7 | **LotConfig** | Lot configuration | 5 | 0 | 1460 |
| 8 | **LandSlope** | Slope of property | 3 | 0 | 1460 |
| 9 | **Neighborhood** | Physical locations within Ames city limits | 25 | 0 | 1460 |
| 10 | **Condition1** | Proximity to various conditions | 9 | 0 | 1460 |
| 11 | **Condition2** | Proximity to various conditions (if more than one is present) | 8 | 0 | 1460 |
| 12 | **BldgType** | Type of dwelling | 5 | 0 | 1460 |
| 13 | **HouseStyle** | Style of dwelling | 8 | 0 | 1460 |
| 14 | **RoofStyle** | Type of roof | 6 | 0 | 1460 |
| 15 | **RoofMatl** | Roof material | 8 | 0 | 1460 |
| 16 | **Exterior1st** | Exterior covering on house | 15 | 0 | 1460 |
| 17 | **Exterior2nd** | Exterior covering on house (if more than one material) | 16 | 0 | 1460 |
| 18 | **MasVnrType** | Masonry veneer type | 4 | 8 | 1452 |
| 19 | **ExterQual** | Evaluates the quality of the material on the exterior | 4 | 0 | 1460 |
| 20 | **ExterCond** | Evaluates the present condition of the material on the exterior | 5 | 0 | 1460 |
| 21 | **Foundation** | Type of foundation | 6 | 0 | 1460 |
| 22 | **BsmtQual** | Evaluates the height of the basement | 4 | 37 | 1423 |
| 23 | **BsmtCond** | Evaluates the general condition of the basement | 4 | 37 | 1423 |
| 24 | **BsmtExposure** | Refers to walkout or garden level walls | 4 | 38 | 1422 |
| 25 | **BsmtFinType1** | Rating of basement finished area | 6 | 37 | 1423 |
| 26 | **BsmtFinType2** | Rating of basement finished area (if multiple types) | 6 | 38 | 1422 |
| 27 | **Heating** | Type of heating | 6 | 0 | 1460 |
| 28 | **HeatingQC** | Heating quality and condition | 5 | 0 | 1460 |
| 29 | **CentralAir** | Central air conditioning | 2 | 0 | 1460 |
| 30 | **Electrical** | Electrical system | 5 | 1 | 1459 |
| 31 | **KitchenQual** | Kitchen quality | 4 | 0 | 1460 |
| 32 | **Functional** | Home functionality (Assume typical unless deductions are warranted) | 7 | 0 | 1460 |
| 33 | **FireplaceQu** | Fireplace quality | 5 | 690 | 770 |
| 34 | **GarageType** | Garage location | 6 | 81 | 1379 |
| 35 | **GarageFinish** | Interior finish of the garage | 3 | 81 | 1379 |
| 36 | **GarageQual** | Garage quality | 5 | 81 | 1379 |
| 37 | **GarageCond** | Garage condition | 5 | 81 | 1379 |
| 38 | **PavedDrive** | Paved driveway | 3 | 0 | 1460 |
| 39 | **PoolQC** | Pool quality | 3 | 1453 | 7 |
| 40 | **Fence** | Fence quality | 4 | 1179 | 281 |
| 41 | **MiscFeature** | Miscellaneous feature not covered in other categories | 4 | 1406 | 54 |
| 42 | **SaleType** | Type of sale | 9 | 0 | 1460 |
| 43 | **SaleCondition** | Condition of sale | 6 | 0 | 1460 |


Наиболее интересными выглядят признаки `Alley`, `FireplaceQu`, `PoolQC`, `Fence`, `MiscFeature`, т.к. много записей в данных, где эти признаки не определены. Возможно, это наиболее дорогое или наоборот наиболее дешёвое жильё.

In [353]:
def report(feature_name: str) -> None:
    not_na_records = train[~train[feature_name].isna()]
    na_records = train[train[feature_name].isna()]

    alley_not_na_percentile = ''.join([f'    - {x}: {np.percentile(not_na_records["SalePrice"], x)}\n' for x in [5, 25, 50, 75, 95]])
    alley_na_percentile = ''.join([f'    - {x}: {np.percentile(na_records["SalePrice"], x)}\n' for x in [5, 25, 50, 75, 95]])

    display_markdown(f"""
##### `{feature_name}` == n/a
- Записей `{feature_name}` == n/a: {na_records.shape[0]}
- Распределение стоимости жилья (перцентиль):

{alley_na_percentile}

##### `{feature_name}` != n/a
- Записей `{feature_name}` != n/a: {not_na_records.shape[0]}
- Распределение стоимости жилья (перцентиль):

{alley_not_na_percentile}
""", raw=True)

def ks(feature_name: str) -> None:
    result = ks_2samp(train[~train[feature_name].isna()]['SalePrice'], train[train[feature_name].isna()]['SalePrice'])
    return (result.statistic, result.pvalue) 

def hist(feature_name: str, nbins: int=100, log: bool=False) -> None:
    fig = go.Figure()

    na_data = train[train[feature_name].isna()]['SalePrice'].apply(lambda x: np.log(x) if log else x)
    not_na_data = train[~train[feature_name].isna()]['SalePrice'].apply(lambda x: np.log(x) if log else x)

    fig.add_trace(go.Histogram(x=na_data, nbinsx=nbins, name=f'{feature_name} is na'))
    fig.add_trace(go.Histogram(x=not_na_data, nbinsx=nbins, name=f'{feature_name} is not na'))

    fig.update_layout(showlegend=True, height=400, width=1000, title=feature_name)
    fig.show()

#### Alley

In [335]:
report('Alley')


##### `Alley` == n/a
- Записей `Alley` == n/a: 1369
- Распределение стоимости жилья (перцентиль):

    - 5: 89188.4
    - 25: 130500.0
    - 50: 165000.0
    - 75: 217500.0
    - 95: 335000.0


##### `Alley` != n/a
- Записей `Alley` != n/a: 91
- Распределение стоимости жилья (перцентиль):

    - 5: 70000.0
    - 25: 114752.0
    - 50: 140000.0
    - 75: 172200.0
    - 95: 207950.0



Из данных выше сложно делать какие-то выводы. Проведём тест Колмогорова-Смирнова:

In [336]:
stat, _ = ks('Alley')
print(stat)

0.2560383371194182


Значение статистики большое &mdash; можно отвергнуть нулевую гипотезу, что данные имеют одинаковое распределение.

Изобразим данные в разрезе признака на графике:

In [354]:
hist('Alley')

График так же не даёт возможности сделать однозначные выводы.

#### FireplaceQu

In [338]:
report('FireplaceQu')


##### `FireplaceQu` == n/a
- Записей `FireplaceQu` == n/a: 690
- Распределение стоимости жилья (перцентиль):

    - 5: 80000.0
    - 25: 112000.0
    - 50: 135000.0
    - 75: 164375.0
    - 95: 224955.0


##### `FireplaceQu` != n/a
- Записей `FireplaceQu` != n/a: 770
- Распределение стоимости жилья (перцентиль):

    - 5: 119335.0
    - 25: 159000.0
    - 50: 191000.0
    - 75: 257375.0
    - 95: 380550.0



Кажется, что жильё с не-пустым `FireplaceQu` стоит дороже. Проведём тест Колмогорова-Смирнова:

In [339]:
ks('FireplaceQu')

(0.4862224731789949, 8.575157499901099e-80)

Значение статистики большое &mdash; можно отвергнуть нулевую гипотезу, что данные имеют одинаковое распределение.

Изобразим данные в разрезе признака на графике:

In [355]:
hist('FireplaceQu', log=True, nbins=50)
hist('FireplaceQu')

На графике видно, что цены на жильё с `FireplaceQu` выше.

#### Fence

In [350]:
report('Fence')


##### `Fence` == n/a
- Записей `Fence` == n/a: 1179
- Распределение стоимости жилья (перцентиль):

    - 5: 87950.0
    - 25: 132750.0
    - 50: 173000.0
    - 75: 224700.0
    - 95: 339775.0


##### `Fence` != n/a
- Записей `Fence` != n/a: 281
- Распределение стоимости жилья (перцентиль):

    - 5: 89500.0
    - 25: 123000.0
    - 50: 141500.0
    - 75: 166000.0
    - 95: 256000.0



Из данных выше сложно сделать какие-то выводы. Провердём тест Колмогорова-Смирнова:

In [351]:
ks('Fence')

(0.29453454432400944, 1.1102230246251565e-16)

Значение статистики большое &mdash; можно отвергнуть нулевую гипотезу, что данные имеют одинаковое распределение.

Изобразим данные в разрезе признака на графике:

In [356]:
hist('Fence', log=True, nbins=50)
hist('Fence')

Визуально сильных перекосов в данных не заметно.

#### MiscFeature

In [357]:
report('MiscFeature')


##### `MiscFeature` == n/a
- Записей `MiscFeature` == n/a: 1406
- Распределение стоимости жилья (перцентиль):

    - 5: 89500.0
    - 25: 130000.0
    - 50: 164250.0
    - 75: 215000.0
    - 95: 332101.0


##### `MiscFeature` != n/a
- Записей `MiscFeature` != n/a: 54
- Распределение стоимости жилья (перцентиль):

    - 5: 78450.0
    - 25: 120125.0
    - 50: 146000.0
    - 75: 181750.0
    - 95: 259674.99999999994



Данных для не-пустых `MiscFeature` довольно мало, но судя по ним радикальной разницы не заметно. Проведём тест Колмогорова-Смирнова: 

In [359]:
ks('MiscFeature')

(0.1994889626468574, 0.02723654325436331)

Значение статистики большое &mdash; можно отвергнуть нулевую гипотезу, что распределения одинаковые.

Изобразим данные на графике:

In [360]:
hist('MiscFeature', log=True, nbins=50)
hist('MiscFeature')

Данных мало и на графике сложно что-то сказать, но явных различий не видно.

#### PoolQC

`PoolQC` &mdash; качество бассейна на объекте.

Записей с не-пустым `PoolQC` исчезающе мало &mdash; всего 7 штук:

In [373]:
train[~train['PoolQC'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,_t
197,198,75,RL,174.0,25419,Pave,NaN,Reg,Lvl,AllPub,...,Ex,GdPrv,NaN,0,3,2006,WD,Abnorml,235000,train
810,811,20,RL,78.0,10140,Pave,NaN,Reg,Lvl,AllPub,...,Fa,GdPrv,NaN,0,1,2006,WD,Normal,181000,train
1170,1171,80,RL,76.0,9880,Pave,NaN,Reg,Lvl,AllPub,...,Gd,GdPrv,NaN,0,7,2008,WD,Normal,171000,train
1182,1183,60,RL,160.0,15623,Pave,NaN,IR1,Lvl,AllPub,...,Ex,MnPrv,NaN,0,7,2007,WD,Abnorml,745000,train
1298,1299,60,RL,313.0,63887,Pave,NaN,IR3,Bnk,AllPub,...,Gd,NaN,NaN,0,1,2008,New,Partial,160000,train
1386,1387,60,RL,80.0,16692,Pave,NaN,IR1,Lvl,AllPub,...,Fa,MnPrv,TenC,2000,7,2006,WD,Normal,250000,train
1423,1424,80,RL,NaN,19690,Pave,NaN,IR1,Lvl,AllPub,...,Gd,GdPrv,NaN,0,8,2006,WD,Alloca,274970,train


In [374]:
report('PoolQC')


##### `PoolQC` == n/a
- Записей `PoolQC` == n/a: 1453
- Распределение стоимости жилья (перцентиль):

    - 5: 88000.0
    - 25: 129900.0
    - 50: 162900.0
    - 75: 213500.0
    - 95: 325774.39999999997


##### `PoolQC` != n/a
- Записей `PoolQC` != n/a: 7
- Распределение стоимости жилья (перцентиль):

    - 5: 163300.0
    - 25: 176000.0
    - 50: 235000.0
    - 75: 262485.0
    - 95: 603990.9999999997



In [376]:
ks('PoolQC')

(0.48382656572608396, 0.050548351243565715)

In [378]:
hist('PoolQC', log=True, nbins=50)
hist('PoolQC')

Выглядит так, что дома с бассейнами смещены ближе к правой границе цен.

#### MSSubClass

In [182]:
print(descriptions['MSSubClass']['description'])
px.histogram(train.append(test), x='MSSubClass', color='_t')

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES




В целом распределение классов `MSSubClass` в обучающем и тестовом наборе приблизительно совпадают, но в обучающем наборе нет ни одной записи с классом `150` ("2-STORY PUD - 1946 & NEWER"). В тестовом наборе есть единственная такая запись:

In [71]:
test[test['MSSubClass'] == 150]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,_t
1358,2819,150,RL,NaN,1700,Pave,NaN,Reg,HLS,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,test


#### MSZoning

In [181]:
print(descriptions['MSZoning']['description'])
px.histogram(train.append(test), x='MSZoning', color='_t')

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density
	



#### LotFrontage

In [189]:
print(descriptions['LotFrontage']['description'])
px.histogram(train.append(test), x='LotFrontage', color='_t', nbins=50)

LotFrontage: Linear feet of street connected to property




In [204]:
train[train.select_dtypes([np.int64, np.float64]).columns[1:-1]].corrwith(train['SalePrice']).sort_values(ascending=False)

OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
dtype: float64

In [219]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{}, {}],
           [{}, {}]],
    subplot_titles=("First Subplot","Second Subplot", "Third Subplot", 'Forth Subplot'))

data = train.append(test)

fig.add_trace(go.Histogram(x=data[data['_t'] == 'train']['MSSubClass']),
                 row=1, col=1)
fig.add_trace(go.Histogram(x=data[data['_t'] == 'test']['MSSubClass']),
                 row=1, col=1)

# fig.add_trace(go.Scatter(x=[1, 2], y=[1, 2]),
#                  row=1, col=2)
# fig.add_trace(go.Scatter(x=[1, 2, 3], y=[2, 1, 2]),
#                  row=2, col=1)

fig.update_layout(showlegend=False, barmode='stack', height=800)
fig.show()
# data.head()

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', '_t'],
      dtype='object')